In [1]:
using QuantumOptics
using PyPlot
PyPlot.pygui(true)

true

In [666]:
# Parameters
γ2 = sqrt(2pi * 0.182) # in MHz, intermediate state decay to |1>
γ3 = sqrt(0.01) # in MHz, Rydberg decay to |4> (some other states we don't care about). 0.01MHz ~ 100us

beamWaistOutofPlane = 15
beamWaistInPlane = 600
radiusofArray = 400


Ω1 = 5.5*sqrt(1.73/0.018*15/beamWaistInPlane*15/beamWaistOutofPlane) * 2pi     # in MHz
Ω2 = 5.5*sqrt(1.73/0.018*15/beamWaistInPlane*15/beamWaistOutofPlane) * 2pi # in MHz

Ω1Edge = Ω1*sqrt(exp(-2*(radiusofArray/beamWaistInPlane)^2))
Ω2Edge = Ω2*sqrt(exp(-2*(radiusofArray/beamWaistInPlane)^2))


Δ = 0     # in MHz
tmax = 5  # in μs
dt = 0.001
tlist = [0:dt:tmax;]

5001-element Vector{Float64}:
 0.0
 0.001
 0.002
 0.003
 0.004
 0.005
 0.006
 0.007
 0.008
 0.009
 ⋮
 4.992
 4.993
 4.994
 4.995
 4.996
 4.997
 4.998
 4.999
 5.0

In [667]:
# Basis and operators
b = NLevelBasis(4)
σ1 = transition(b, 1, 2)  # |1><2|
σ2 = transition(b, 2, 3)  # |2><3|
Ryd = transition(b, 4, 3)  # |4><3|, decay out of the system
proj1 = transition(b, 1, 1) # |1><1|
proj2 = transition(b, 2, 2) # |2><2|
proj3 = transition(b, 3, 3)     # |3><3|

Operator(dim=4x4)
  basis: NLevel(N=4)sparse([3], [3], ComplexF64[1.0 + 0.0im], 4, 4)

In [668]:
# Hamiltonian and jump operators
J = [γ2*σ1+γ3*Ryd];
Jdagger = [γ2*dagger(σ1)+γ3*dagger(Ryd)]; 

# Initial state
ψ0 = nlevelstate(b, 1)

Ket(dim=4)
  basis: NLevel(N=4)
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [859]:
# Define the Gaussian function
function gaussian(t, mu, sigma, A)
    return A * exp(-(t - mu)^2 / (2 * sigma^2))
end

mu1 = 2.8  # center of the peak in us
mu2 = 2  # center of the peak in us
sigma = 0.45 # span of in us
tau = 0.01  # time resolution in us

# Define the driving term functions for the center
function H(t, psi)
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return 0.5*(σ1+dagger(σ1))*gaussian(t_rounded, mu1, sigma, Ω1)+0.5*(σ2+dagger(σ2))*gaussian(t_rounded, mu2, sigma, Ω2)+Δ*proj2+1im*γ3*proj3, J, Jdagger
end

# h_mu2 = H(mu2,ψ0)

# Plots
figure(figsize=(6, 3))
plot(tlist, gaussian.(tlist, mu1, sigma, Ω1), label="1-2")
plot(tlist, gaussian.(tlist, mu2, sigma, Ω2), label="2-3")
axis([0, tmax, 0, Ω1])
legend()
PyPlot.show(block=false)

# Define the driving term functions for the edge of the array
function HEdge(t, psi)
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return 0.5*(σ1+dagger(σ1))*gaussian(t_rounded, mu1, sigma, Ω1Edge)+0.5*(σ2+dagger(σ2))*gaussian(t_rounded, mu2, sigma, Ω2Edge)+Δ*proj2+1im*γ3*proj3, J, Jdagger
end

#=
function H_pump2(t, psi)
    mu = 3e-6  # center of the peak at 3us
    sigma = 1e-6  # span of 1us
    tau = 1e-9  # time resolution of 1ns
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return dagger(σ2)*gaussian(t_rounded, mu, sigma, Ω2)
end
=#

HEdge (generic function with 1 method)

In [860]:
# Expectation values
function calc_pops(t, ρ)
    p1 = real(expect(proj1, ρ))
    p2 = real(expect(proj2, ρ))
    p3 = real(expect(proj3, ρ))
    return p1, p2, p3
end

calc_pops (generic function with 1 method)

In [861]:
# Time evolution
tout, pops = timeevolution.master_dynamic(tlist, ψ0, H; fout=calc_pops)
toutEdge, popsEdge = timeevolution.master_dynamic(tlist, ψ0, HEdge; fout=calc_pops)

([0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009  …  4.991, 4.992, 4.993, 4.994, 4.995, 4.996, 4.997, 4.998, 4.999, 5.0], Tuple{Any, Any, Any}[(1.0, 0.0, 0.0), (1.0, 1.544300660302515e-20, 3.3699663499700983e-28), (1.0, 6.057782606685151e-20, 1.3083689185843527e-27), (1.0, 1.3367409932550213e-19, 2.8560399310367935e-27), (1.0, 2.330827225821886e-19, 4.923732102469491e-27), (1.0, 3.572358473093258e-19, 7.456967212837994e-27), (1.0, 5.0464688024851745e-19, 1.0403066391938237e-26), (1.0, 6.739104834105527e-19, 1.3711150119406558e-26), (1.0, 8.637025740754332e-19, 1.7332138224719678e-26), (1.0, 1.0727803247923415e-18, 2.1218749887194706e-26)  …  (0.0036574032997914747, 0.00053980966411973, 0.9703797832680727), (0.003658020158447953, 0.0005391928054397239, 0.9703700795187824), (0.0036586363134100317, 0.0005385766504546578, 0.9703603758665285), (0.003659251765491734, 0.0005379611983505184, 0.9703506723113101), (0.0036598665155065614, 0.000537346448313814, 0.97034096885312

In [862]:
# Reshape pops
p1 = [p[1] for p=pops]
p2 = [p[2] for p=pops]
p3 = [p[3] for p=pops]
final_p3 = round(maximum(p3), digits=4)
final_p3_str = string(final_p3)
# Plots
figure(figsize=(6, 3))
plot(tout, p1, label="Initial ground state")
plot(tout, p2, "--", label="Intermediate state")
plot(tout, p3, label="Excited state")
title("Maximum p3 Center: $final_p3_str")
axis([0, tmax, 0, 1])
legend()
PyPlot.show(block=false)

In [863]:
# Reshape pops
p1Edge = [p[1] for p=popsEdge]
p2Edge = [p[2] for p=popsEdge]
p3Edge = [p[3] for p=popsEdge]
final_p3_Edge = round(maximum(p3Edge), digits=4)
final_p3_str_Edge = string(final_p3_Edge)
# Plots
figure(figsize=(6, 3))
plot(toutEdge, p1Edge, label="Initial ground state")
plot(toutEdge, p2Edge, "--", label="Intermediate state")
plot(toutEdge, p3Edge, label="Excited state")
title("Maximum p3 Edge: $final_p3_str_Edge")
axis([0, tmax, 0, 1])
legend()
PyPlot.show(block=false)